In [1]:
import os
import string
import pandas as pd
from dotenv import load_dotenv
from opensearchpy import OpenSearch

ModuleNotFoundError: No module named 'opensearchpy'

In [ ]:
load_dotenv()

In [ ]:
host = [{'host': os.environ.get('OPENSEARCH_HOST'), 'port': os.environ.get('OPENSEARCH_PORT')}]
auth = (os.environ.get('OPENSEARCH_USER'), os.environ.get('OPENSEARCH_PASS'))

client = OpenSearch(
    hosts = [{'host': os.environ.get('OPENSEARCH_HOST'), 'port': os.environ.get('OPENSEARCH_PORT')}],
    http_auth = (os.environ.get('OPENSEARCH_USER'), os.environ.get('OPENSEARCH_PASS')),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    timeout = 999999
)

In [ ]:
client.info()

In [ ]:
docs_index = 'law_analyzer_new5'
docs_index_2 = 'law_analyzer_msib'
docs_index_3 = 'law_analyzer_msib2'

excluded_fields = ["Blocks.additional_context.vector",
                   "Blocks.additional_context.token_size",
                   "Blocks.additional_context.source_token_length",
                   "Blocks.source_token_length",
                   "Blocks.chunks.main_vector", 
                   "Blocks.chunks.context_vector",
                   "Blocks.chunks.context_token_size",
                   "Blocks.chunks.main_token_size",
                   "Blocks.chunks.source_token_length",]

excluded_fields_2 = ["Blocks.ContentText.MainVector", 
                     "Blocks.ContentText.AdditionalContext.Vector",]

In [ ]:
def query_by_id(doc_id, exc_field):
    return {
        "_source":{
            "excludes": exc_field
        },
        "query": {
            "ids": {
                "values": [doc_id]
            }
        }
    }

def get_doc_df(docs_idx, doc_id, exc_field):
    doc = client.search(index=docs_idx, body=query_by_id(doc_id, exc_field))
    return pd.json_normalize(doc['hits']['hits'])

def get_docs_df(docs_idx, doc_ids, exc_field):
    docs = []
    for doc_id in doc_ids:
        docs.append(get_doc_df(docs_idx, doc_id, exc_field))
    return pd.concat(docs)

In [ ]:
doc_count = client.count(index=docs_index_3)['count']
doc_count

In [ ]:
ids = client.search(index=docs_index_3, body={"query": {"match_all": {}}}, _source=False, size=doc_count)['hits']['hits']
ids = [id['_id'] for id in ids]

In [10]:
ids

['76-pmk.03-2010',
 '227-pmk.05-2011',
 '38-pmk.05-2012',
 '149-pmk.03-2011',
 '20-pmk.04-2018',
 'perpres-44-tahun-2013',
 '35-pmk.07-2013',
 '185-pmk.07-2013',
 '133-pmk.01-2017',
 '230-pmk.07-2014',
 '187-pmk.07-2013',
 'pp-4-tahun-2013',
 '69-pmk.02-2010',
 '225-pmk.011-2013',
 '100-pmk.05-2016',
 '107-pmk.010-2015',
 '76-pmk.07-2014',
 '90-pmk.01-2013',
 '97-pmk.011-2012',
 '185-pmk.06-2009',
 '25-pmk.01-2014',
 '176-pmk.05-2017',
 'perpres-16-tahun-2015',
 '168-pmk.06-2013',
 '228-pmk.010-2012',
 'pp-45-tahun-2014',
 'perpres-85-tahun-2013',
 '165-pmk.01-2016',
 'pp-57-tahun-2013',
 'pp-85-tahun-2015',
 '82-pmk.03-2017',
 '17-pmk.03-2011',
 'perpres-31-tahun-2013',
 '13-pmk.07-2010',
 '46-pmk.08-2016',
 '222-pmk.07-2017',
 'pp-32-tahun-2014',
 '132-pmk.02-2019',
 '38-pmk.011-2013',
 '125-pmk.011-2012',
 '85-pmk.010-2016',
 '195-pmk.02-2017',
 '48-pmk.04-2012',
 'pp-85-tahun-2013',
 '93-pmk.01-2018',
 'pp-59-tahun-2012',
 '84-pmk.05-2013',
 '174-pmk.03-2015',
 '10-pmk.03-2013',
 '

In [11]:
len(ids)

243

In [13]:
docs_df = get_docs_df(docs_index_3, ids, excluded_fields_2)
docs_df

,_index,_id,_score,_source.Status,_source.No,_source.PeraturanGoId,_source.Tematik,_source.Blocks,_source.TanggalPenetapan,_source.Bidang,_source.Slug,_source.Source,_source.Nomor,_source.BlocksMinioPath,_source.FileMinioPath,_source.Judul,_source.Tahun,_source.TanggalPengundangan,_source.Bentuk,_source.PeraturanId
0,law_analyzer_msib2,76-pmk.03-2010,1.0,Tidak Berlaku,76,https://peraturan.go.id/files/bn159-2010.pdf,[Unclassified],"[{'ContentText': [{'AdditionalContext': [], 'R...",2010-03-31 00:00:00.000,,76/pmk.03/2010,jdih.kemenkeu.go.id,76/PMK.03/2010,law_analyzer_parse/01956C5F-E27F-413E-BC82-A5B...,law_analyzer/01956C5F-E27F-413E-BC82-A5B879E81...,Tata Cara Pengajuan dan Penyelesaian Permintaa...,2010,2010-03-31 00:00:00.000,Peraturan Menteri Keuangan,01956C5F-E27F-413E-BC82-A5B879E81125
0,law_analyzer_msib2,227-pmk.05-2011,1.0,Berlaku,227,https://peraturan.go.id/files/bn856-2011.pdf,[Unclassified],"[{'ContentText': [{'AdditionalContext': [], 'R...",2011-12-19 00:00:00.000,Hukum Keuangan Negara,227/pmk.05/2011,jdih.kemenkeu.go.id,227/PMK.05/2011,law_analyzer_parse/0DD677AE-42DD-4AE7-8953-71F...,law_analyzer/0DD677AE-42DD-4AE7-8953-71F519FAB...,Penyusunan dan Pengesahan Daftar Isian Pelaksa...,2011,2011-12-19 00:00:00.000,Peraturan Menteri Keuangan,0DD677AE-42DD-4AE7-8953-71F519FAB13C
0,law_analyzer_msib2,38-pmk.05-2012,1.0,Tidak Berlaku,38,https://peraturan.go.id/files/bn289-2012.pdf,[Unclassified],"[{'ContentText': [{'AdditionalContext': [], 'R...",2012-03-12 00:00:00.000,,38/pmk.05/2012,jdih.kemenkeu.go.id,38/PMK.05/2012,law_analyzer_parse/611AD6BB-D0CF-41F0-BFF3-350...,law_analyzer/611AD6BB-D0CF-41F0-BFF3-350BF669A...,Tarif Layanan Badan Layanan Umum Institut Agam...,2012,2012-03-12 00:00:00.000,Peraturan Menteri Keuangan,611AD6BB-D0CF-41F0-BFF3-350BF669ABA1
0,law_analyzer_msib2,149-pmk.03-2011,1.0,Berlaku,149,https://peraturan.go.id/files/bn573-2011.pdf,[Unclassified],"[{'ContentText': [{'AdditionalContext': [], 'R...",2011-09-12 00:00:00.000,Hukum Keuangan Negara,149/pmk.03/2011,jdih.kemenkeu.go.id,149/PMK.03/2011,law_analyzer_parse/3413A28E-D9D1-49E8-9AC6-C64...,law_analyzer/3413A28E-D9D1-49E8-9AC6-C64022DAC...,Sensus Pajak Nasional.,2011,2011-09-12 00:00:00.000,Peraturan Menteri Keuangan,3413A28E-D9D1-49E8-9AC6-C64022DACD32
0,law_analyzer_msib2,20-pmk.04-2018,1.0,Tidak Berlaku,20,https://peraturan.go.id/files/bn320-2018.pdf,[Unclassified],[{'ContentText': [{'AdditionalContext': [{'Tex...,2018-02-26 00:00:00.000,,20/pmk.04/2018,jdih.kemenkeu.go.id,20/PMK.04/2018,law_analyzer_parse/D3826423-5898-4C28-A8D8-267...,law_analyzer/D3826423-5898-4C28-A8D8-2670E7714...,Perubahan atas Peraturan Menteri Keuangan Nomo...,2018,2018-02-26 00:26:00.000,Peraturan Menteri Keuangan,D3826423-5898-4C28-A8D8-2670E77149E8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,law_analyzer_msib2,pp-81-tahun-2012,1.0,Berlaku,81,https://peraturan.go.id/files/pp81-2012bt.pdf,[Unclassified],"[{'ContentText': [{'AdditionalContext': [], 'R...",2012-10-12 00:00:00.000,,pp-81-tahun-2012,jdih.kemenkeu.go.id,PP 81 TAHUN 2012,law_analyzer_parse/490A3E49-26EC-4CEB-A0B6-CE3...,law_analyzer/490A3E49-26EC-4CEB-A0B6-CE37BCBD9...,Pengelolaan Sampah Rumah Tangga dan Sampah Sej...,2012,2012-10-12 00:00:00.000,Peraturan Pemerintah,490A3E49-26EC-4CEB-A0B6-CE37BCBD961E
0,law_analyzer_msib2,172-pmk.08-2017,1.0,Tidak Berlaku,172,https://peraturan.go.id/files/bn1679-2017.pdf,[Unclassified],"[{'ContentText': [{'AdditionalContext': [], 'R...",2017-11-23 00:00:00.000,,172/pmk.08/2017,jdih.kemenkeu.go.id,172/PMK.08/2017,law_analyzer_parse/07EF544E-9C1A-4C34-BE97-3DF...,law_analyzer/07EF544E-9C1A-4C34-BE97-3DF4B2DC1...,Tata Cara Pengadaan Pinjaman Luar Negeri Tunai.,2017,2017-11-23 00:00:00.000,Peraturan Menteri Keuangan,07EF544E-9C1A-4C34-BE97-3DF4B2DC1A12
0,law_analyzer_msib2,90-pmk.05-2009,1.0,Berlaku,90,https://peraturan.go.id/files/bn98-2009.pdf,[Unclassified],"[{'ContentText': [{'AdditionalContext': [], 'R...",2009-05-08 00:00:00.0

In [14]:
# save the dataframe to csv
docs_df.to_csv('data/law_analyzer_msib2.csv', index=False)

In [19]:
docs_df = pd.read_csv('data/law_analyzer_msib2.csv')
docs_df

,_index,_id,_score,_source.Status,_source.No,_source.PeraturanGoId,_source.Tematik,_source.Blocks,_source.TanggalPenetapan,_source.Bidang,_source.Slug,_source.Source,_source.Nomor,_source.BlocksMinioPath,_source.FileMinioPath,_source.Judul,_source.Tahun,_source.TanggalPengundangan,_source.Bentuk,_source.PeraturanId
0,law_analyzer_msib2,76-pmk.03-2010,1.0,Tidak Berlaku,76,https://peraturan.go.id/files/bn159-2010.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2010-03-31 00:00:00.000,NaN,76/pmk.03/2010,jdih.kemenkeu.go.id,76/PMK.03/2010,law_analyzer_parse/01956C5F-E27F-413E-BC82-A5B...,law_analyzer/01956C5F-E27F-413E-BC82-A5B879E81...,Tata Cara Pengajuan dan Penyelesaian Permintaa...,2010,2010-03-31 00:00:00.000,Peraturan Menteri Keuangan,01956C5F-E27F-413E-BC82-A5B879E81125
1,law_analyzer_msib2,227-pmk.05-2011,1.0,Berlaku,227,https://peraturan.go.id/files/bn856-2011.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2011-12-19 00:00:00.000,Hukum Keuangan Negara,227/pmk.05/2011,jdih.kemenkeu.go.id,227/PMK.05/2011,law_analyzer_parse/0DD677AE-42DD-4AE7-8953-71F...,law_analyzer/0DD677AE-42DD-4AE7-8953-71F519FAB...,Penyusunan dan Pengesahan Daftar Isian Pelaksa...,2011,2011-12-19 00:00:00.000,Peraturan Menteri Keuangan,0DD677AE-42DD-4AE7-8953-71F519FAB13C
2,law_analyzer_msib2,38-pmk.05-2012,1.0,Tidak Berlaku,38,https://peraturan.go.id/files/bn289-2012.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2012-03-12 00:00:00.000,NaN,38/pmk.05/2012,jdih.kemenkeu.go.id,38/PMK.05/2012,law_analyzer_parse/611AD6BB-D0CF-41F0-BFF3-350...,law_analyzer/611AD6BB-D0CF-41F0-BFF3-350BF669A...,Tarif Layanan Badan Layanan Umum Institut Agam...,2012,2012-03-12 00:00:00.000,Peraturan Menteri Keuangan,611AD6BB-D0CF-41F0-BFF3-350BF669ABA1
3,law_analyzer_msib2,149-pmk.03-2011,1.0,Berlaku,149,https://peraturan.go.id/files/bn573-2011.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2011-09-12 00:00:00.000,Hukum Keuangan Negara,149/pmk.03/2011,jdih.kemenkeu.go.id,149/PMK.03/2011,law_analyzer_parse/3413A28E-D9D1-49E8-9AC6-C64...,law_analyzer/3413A28E-D9D1-49E8-9AC6-C64022DAC...,Sensus Pajak Nasional.,2011,2011-09-12 00:00:00.000,Peraturan Menteri Keuangan,3413A28E-D9D1-49E8-9AC6-C64022DACD32
4,law_analyzer_msib2,20-pmk.04-2018,1.0,Tidak Berlaku,20,https://peraturan.go.id/files/bn320-2018.pdf,['Unclassified'],[{'ContentText': [{'AdditionalContext': [{'Tex...,2018-02-26 00:00:00.000,NaN,20/pmk.04/2018,jdih.kemenkeu.go.id,20/PMK.04/2018,law_analyzer_parse/D3826423-5898-4C28-A8D8-267...,law_analyzer/D3826423-5898-4C28-A8D8-2670E7714...,Perubahan atas Peraturan Menteri Keuangan Nomo...,2018,2018-02-26 00:26:00.000,Peraturan Menteri Keuangan,D3826423-5898-4C28-A8D8-2670E77149E8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,law_analyzer_msib2,pp-81-tahun-2012,1.0,Berlaku,81,https://peraturan.go.id/files/pp81-2012bt.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2012-10-12 00:00:00.000,NaN,pp-81-tahun-2012,jdih.kemenkeu.go.id,PP 81 TAHUN 2012,law_analyzer_parse/490A3E49-26EC-4CEB-A0B6-CE3...,law_analyzer/490A3E49-26EC-4CEB-A0B6-CE37BCBD9...,Pengelolaan Sampah Rumah Tangga dan Sampah Sej...,2012,2012-10-12 00:00:00.000,Peraturan Pemerintah,490A3E49-26EC-4CEB-A0B6-CE37BCBD961E
239,law_analyzer_msib2,172-pmk.08-2017,1.0,Tidak Berlaku,172,https://peraturan.go.id/files/bn1679-2017.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2017-11-23 00:00:00.000,NaN,172/pmk.08/2017,jdih.kemenkeu.go.id,172/PMK.08/2017,law_analyzer_parse/07EF544E-9C1A-4C34-BE97-3DF...,law_analyzer/07EF544E-9C1A-4C34-BE97-3DF4B2DC1...,Tata Cara Pengadaan Pinjaman Luar Negeri Tunai.,2017,2017-11-23 00:00:00.000,Peraturan Menteri Keuangan,07EF544E-9C1A-4C34-BE97-3DF4B2DC1A12
240,law_analyzer_msib2,90-pmk.05-2009,1.0,Berlaku,90,https://peraturan.go.id/files/bn98-2009.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalConte

In [17]:
# close connection from opensearch
client.close()

In [ ]:
invalid_docs_id = []

for index, row in docs_df.iterrows():
    if type(row['_source.Blocks']) != str:
        invalid_docs_id.append(row['_id'])

invalid_docs_id

In [ ]:
# drop the invalid documents
docs_df = docs_df[~docs_df['_id'].isin(invalid_docs_id)]
docs_df

In [ ]:
# convert all _source.Blocks to list
docs_df['_source.Blocks'] = docs_df['_source.Blocks'].apply(eval)

In [ ]:
content_type_list = []

for index, row in docs_df.iterrows():
    list(row['_source.Blocks'])
    if type(row['_source.Blocks']) == list:
        for k in row['_source.Blocks']:
            if 'type' in k:
                if k['type'] not in content_type_list:
                    content_type_list.append(k['type'])
            else:
                pass

content_type_list

In [ ]:
# create new dataframe with only the relevant fields (_id, _source.Judul, _source.Blocks)
data_df = docs_df[['_id','_source.Blocks']]
data_df

In [ ]:
def context_prep(context_text):
    # remove all characters before "-" and including "-".
    # if context_text contains "-", remove it and the characters before it
    if "-" in context_text:
        text = context_text.split("-", 1)[1].strip()
        text = text.translate(str.maketrans('', '', string.punctuation)).strip()
    else:
        text = context_text.translate(str.maketrans('', '', string.punctuation)).strip()
        ""
    return ' '.join(text.split())


def title_prep(context_text):
    # remove all characters after "-" and including "-"
    # if context_text contains "-", remove it and the characters after it
    if "-" in context_text:
        text = context_text.split("-", 1)[0].strip()
        text = text.translate(str.maketrans('', '', string.punctuation)).strip()
    else:
        text = context_text.translate(str.maketrans('', '', string.punctuation)).strip()

    return ' '.join(text.split())


def context_prep_2(context_text):
    context_parts = context_text.split(" - ")
    # join the parts after the first part
    text = ' - '.join(context_parts[1:]).strip()

    return text


def title_prep_2(context_text):
    context_parts = context_text.split(" - ")

    return context_parts[0].strip()

In [ ]:
docs_title_list = []

valid_docs_content_id = []
invalid_docs_content_id = []

for index, row in data_df.iterrows():
    for c in row['_source.Blocks']:
        if 'type' in c and c['type'] == 'CONTENT_PASAL':
            title_text = title_prep_2(c['context'])
            docs_title_list.append(title_text)
            valid_docs_content_id.append(row['_id'])
            break
        else:
            if row['_id'] not in invalid_docs_content_id:
                invalid_docs_content_id.append(row['_id'])

# remove id in invalid_docs_title_id if exists in valid_docs_title_id
invalid_docs_content_id = [x for x in invalid_docs_content_id if x not in valid_docs_content_id]
invalid_docs_content_id

In [ ]:
docs_title_list

In [ ]:
# delete the invalid documents
data_df = data_df[~data_df['_id'].isin(invalid_docs_content_id)]
data_df

In [ ]:
docs_content_list = []
docs_id_list = []

for index, row in data_df.iterrows():
    doc_content_list = []
    context_temp = []

    for c in row['_source.Blocks']:
        if 'type' in c and c['type'] == 'CONTENT_PASAL':
            title_text = title_prep_2(c['context'])
            context_text = context_prep_2(c['context'])
            
            if context_text not in context_temp:
                doc_content_list.append(f'{context_text} {c["content"]}')
                context_temp.append(context_text)
            else:
                doc_content_list.append(c['content'])

    doc_content_string = ' '.join(doc_content_list)
    docs_content_list.append(doc_content_string)
    docs_id_list.append(row['_id'])

In [ ]:
# create dictionary to map the id to the content
id_to_content = dict(zip(docs_id_list, docs_content_list))
id_to_title = dict(zip(docs_id_list, docs_title_list))

# add the content to the dataframe
data_df['title'] = data_df['_id'].map(id_to_title)
data_df['content'] = data_df['_id'].map(id_to_content)

# delete the _source.Blocks column
data_df = data_df.drop(columns='_source.Blocks')
data_df

In [ ]:
invalid_title_id = []

for index, row in data_df.iterrows():
    if len(row['title']) == 0:
        invalid_title_id.append(row['_id'])

invalid_title_id

In [ ]:
# drop the invalid documents
data_df = data_df[~data_df['_id'].isin(invalid_title_id)]
data_df

In [ ]:
# save the dataframe to csv
data_df.to_csv('data/law_analyzer_new4_title-content.csv', index=False)